In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
df=pd.read_csv(r"E:\colledge_project\skin_c_proj\SKIN\hmnist_28_28_RGB.csv.zip")

In [3]:
df.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351,label
0,192,153,193,195,155,192,197,154,185,202,...,173,124,138,183,147,166,185,154,177,2
1,25,14,30,68,48,75,123,93,126,158,...,60,39,55,25,14,28,25,14,27,2
2,192,138,153,200,145,163,201,142,160,206,...,167,129,143,159,124,142,136,104,117,2
3,38,19,30,95,59,72,143,103,119,171,...,44,26,36,25,12,17,25,12,15,2
4,158,113,139,194,144,174,215,162,191,225,...,209,166,185,172,135,149,109,78,92,2


In [4]:
df.label.value_counts()

label
4    6705
6    1113
2    1099
1     514
0     327
5     142
3     115
Name: count, dtype: int64

In [5]:
X = df.drop('label', axis=1).values.astype(np.float32) / 255.0  # Normalize pixel values
y = df['label'].values.astype(np.int32)

In [6]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X, y = sm.fit_resample(X,y)

In [7]:
# Check for Imbalance using SMOTE
pd.DataFrame(y).value_counts()

0
0    6705
1    6705
2    6705
3    6705
4    6705
5    6705
6    6705
Name: count, dtype: int64

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
img_height, img_width = 28, 28

X_train_reshaped = X_train.reshape(-1, img_height, img_width, 3)
X_test_reshaped = X_test.reshape(-1, img_height, img_width, 3)

In [10]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.Dropout(0.1),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(7, activation='sigmoid')  # 7 output classes
])


C:\Users\abhin\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [12]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

In [13]:
train_generator = datagen.flow(X_train_reshaped, y_train, batch_size=32)
test_generator = datagen.flow(X_test_reshaped, y_test, batch_size=32)

In [14]:
train_generator.x.shape

(37548, 28, 28, 3)

In [15]:
model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // 32,
    epochs=15,
    validation_data=test_generator,
    validation_steps=len(X_test) // 32,
)

Epoch 1/15


C:\Users\abhin\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1173/1173 ━━━━━━━━━━━━━━━━━━━━ 43s 34ms/step - accuracy: 0.3467 - loss: 1.5606 - val_accuracy: 0.5548 - val_loss: 1.1294
Epoch 2/15
1173/1173 ━━━━━━━━━━━━━━━━━━━━ 0s 47us/step - accuracy: 0.5625 - loss: 1.2110 - val_accuracy: 0.3636 - val_loss: 1.5611
Epoch 3/15


c:\Program Files\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


1173/1173 ━━━━━━━━━━━━━━━━━━━━ 32s 27ms/step - accuracy: 0.5621 - loss: 1.0941 - val_accuracy: 0.6294 - val_loss: 0.9580
Epoch 4/15
1173/1173 ━━━━━━━━━━━━━━━━━━━━ 0s 15us/step - accuracy: 0.7188 - loss: 0.6454 - val_accuracy: 0.5455 - val_loss: 0.9772
Epoch 5/15
1173/1173 ━━━━━━━━━━━━━━━━━━━━ 31s 26ms/step - accuracy: 0.6207 - loss: 0.9689 - val_accuracy: 0.6231 - val_loss: 0.9676
Epoch 6/15
1173/1173 ━━━━━━━━━━━━━━━━━━━━ 0s 15us/step - accuracy: 0.5625 - loss: 1.0560 - val_accuracy: 0.7273 - val_loss: 0.5744
Epoch 7/15
1173/1173 ━━━━━━━━━━━━━━━━━━━━ 31s 26ms/step - accuracy: 0.6544 - loss: 0.9006 - val_accuracy: 0.6960 - val_loss: 0.8166
Epoch 8/15
1173/1173 ━━━━━━━━━━━━━━━━━━━━ 0s 14us/step - accuracy: 0.6250 - loss: 0.8382 - val_accuracy: 0.7273 - val_loss: 0.8989
Epoch 9/15
 494/1173 ━━━━━━━━━━━━━━━━━━━━ 15s 23ms/step - accuracy: 0.6690 - loss: 0.8541

In [23]:
test_loss, test_acc = model.evaluate(X_test_reshaped, y_test, verbose=2)
print(f'Test accuracy: {test_acc}')

294/294 - 1s - 2ms/step - accuracy: 0.7744 - loss: 0.6036
Test accuracy: 0.7743688225746155


In [24]:
y_pred = model.predict(X_test_reshaped)

294/294 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [25]:
y_pred = np.argmax(y_pred, axis=1)

In [26]:
pd.DataFrame(y_pred).value_counts()

0
3    1549
0    1515
2    1373
5    1289
6    1286
4    1236
1    1139
Name: count, dtype: int64

In [27]:
model.save("model.h5")

In [28]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

[[1035  117   39   64   10    0   30]
 [ 196  884   69  106   22   14   32]
 [ 133   47  931   29   64    7  140]
 [  46   29    4 1297    3    0   13]
 [  31   34  100    8  982    6  185]
 [   0    6    1    9   16 1257    3]
 [  74   22  229   36  139    5  883]]


# VGG16 Implementation

In [32]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from imblearn.over_sampling import SMOTE

In [43]:
def preprocess_and_resize(image, label):
    image = tf.image.resize(image, [224, 224])
    image = preprocess_input(image)
    return image, label

# One-hot encode labels
y_train_cat = tf.keras.utils.to_categorical(y_train, num_classes=7)
y_test_cat = tf.keras.utils.to_categorical(y_test, num_classes=7)

# Create a TensorFlow Dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_reshaped, y_train_cat))
train_dataset = train_dataset.map(preprocess_and_resize).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

# Create a TensorFlow Dataset for testing
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_reshaped, y_test_cat))
test_dataset = test_dataset.map(preprocess_and_resize).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

# Load the VGG16 model without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add a new top layer for our 7 classes
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(7, activation='softmax')(x)

# Create the new model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_dataset,
    steps_per_epoch=len(X_train) // 32,
    epochs=15,
    validation_data=test_dataset,
    validation_steps=len(X_test) // 32,
)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {test_accuracy}')

# Predict and evaluate
y_pred = model.predict(test_dataset)
y_pred_classes = np.argmax(y_pred, axis=1)

# Print confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(np.argmax(y_test_cat, axis=1), y_pred_classes))

# Save the model
model.save("vgg16_skin_cancer_model.h5")



9

Epoch 1/15
 719/1173 ━━━━━━━━━━━━━━━━━━━━ 2:26:38 19s/step - accuracy: 0.1452 - loss: 2.4095